In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from ipykernel.kernelapp import kernel_aliases
from pySmartDL.download import download
from tensorflow.python.ops.gen_dataset_ops import padded_batch_dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [15]:
# Hyperparameters
input_size = 28       # Each row of the image (28 pixels)
sequence_length = 28  # Number of rows (28 steps)
hidden_size = 128     # Size of hidden state
num_layers = 2        # Number of RNN layers
num_classes = 10      # Digits 0-9
batch_size = 64
num_epochs = 2
learning_rate = 0.001

In [16]:
device=torch.device("cuda" if torch.cuda.is_available() else "cup")

In [17]:
# MNIST dataset
train =datasets.MNIST("",train=True,download=True,
                      transform= transforms.Compose([transforms.ToTensor()]))

test =datasets.MNIST("",train=False,download=True,
                      transform= transforms.Compose([transforms.ToTensor()]))

# Data loaders
train_loader = DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(dataset=test,  batch_size=batch_size, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:38<00:00, 255kB/s] 


Extracting MNIST\raw\train-images-idx3-ubyte.gz to MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 69.1kB/s]


Extracting MNIST\raw\train-labels-idx1-ubyte.gz to MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:15<00:00, 105kB/s] 


Extracting MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.34MB/s]

Extracting MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST\raw



In [20]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# RNN Model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        # Initial hidden state of shape (num_layers, batch_size, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)  # out: (batch_size, seq_length, hidden_size)

        # Flatten the output                                                    #Input x.shape: [2, 28, 28]
                                                                                #Output out.shape: [2, 28, 128]
        out = out.reshape(out.size(0), -1)  # (batch_size, 128*28)
        out = self.fc(out)
        return out

In [21]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move to device
        images = images.reshape(-1, sequence_length, input_size).to(device)  # [batch, 28, 28]
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")


Epoch [1/2], Step [100/938], Loss: 0.4282
Epoch [1/2], Step [200/938], Loss: 0.2655
Epoch [1/2], Step [300/938], Loss: 0.2697
Epoch [1/2], Step [400/938], Loss: 0.1850
Epoch [1/2], Step [500/938], Loss: 0.1492
Epoch [1/2], Step [600/938], Loss: 0.1533
Epoch [1/2], Step [700/938], Loss: 0.1883
Epoch [1/2], Step [800/938], Loss: 0.1232
Epoch [1/2], Step [900/938], Loss: 0.1992
Epoch [2/2], Step [100/938], Loss: 0.2171
Epoch [2/2], Step [200/938], Loss: 0.0966
Epoch [2/2], Step [300/938], Loss: 0.2916
Epoch [2/2], Step [400/938], Loss: 0.1517
Epoch [2/2], Step [500/938], Loss: 0.0344
Epoch [2/2], Step [600/938], Loss: 0.2973
Epoch [2/2], Step [700/938], Loss: 0.1072
Epoch [2/2], Step [800/938], Loss: 0.1842
Epoch [2/2], Step [900/938], Loss: 0.0517


In [22]:
# Testing
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 97.09%


In [ ]:
nice if u are seeeying this you are also fucked